# <font color='#11a642' size='6'> **Построение моделей**

## <font color='#11a642' size='5'> Построение модели **Random Forest**. Гиперпараметры подберите либо вручную, либо с помощью GridSearchCV или RandomizedSearchCV


- необходимо ли обрабатывать категориальные признаки заранее?
- необходимо ли обрабатывать пропуски?

In [ ]:
# ваш код
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Упрощенный поиск параметров
param_grid_rf = {
    'regressor__n_estimators': [50, 100],  # Уменьшили количество деревьев
    'regressor__max_depth': [10, 20],      # Ограничили глубину
    'regressor__min_samples_split': [5, 10]
}

print("Начинаем GridSearch для Random Forest...")
grid_search_rf = GridSearchCV(
    rf_pipeline, 
    param_grid_rf, 
    cv=3,  # Уменьшили количество фолдов
    scoring='neg_mean_squared_error', 
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train, y_train)

# Оценка
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

print("\nRandom Forest Results:")
print(f"MAE: {mean_absolute_error(y_test, y_pred_rf):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_rf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.4f}")
print(f"R2: {r2_score(y_test, y_pred_rf):.4f}")
print(f"Best params: {grid_search_rf.best_params_}")

## <font color='#11a642' size='5'> Построение модели бустинга **XGBoost**. Гиперпараметры подберите либо вручную, либо с помощью GridSearchCV или RandomizedSearchCV


- необходимо ли обрабатывать категориальные признаки заранее?
- необходимо ли обрабатывать пропуски?

In [ ]:
# ваш код
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))
])

# Более реалистичные параметры для XGBoost
param_dist_xgb = {
    'regressor__n_estimators': [100, 200],
    'regressor__learning_rate': [0.1, 0.2],
    'regressor__max_depth': [3, 6],
    'regressor__subsample': [0.8, 1.0]
}

print("Начинаем RandomizedSearch для XGBoost...")
random_search_xgb = RandomizedSearchCV(
    xgb_pipeline, 
    param_dist_xgb, 
    n_iter=8,  # Уменьшили количество итераций
    cv=3, 
    scoring='neg_mean_squared_error', 
    random_state=42, 
    n_jobs=-1,
    verbose=1
)

random_search_xgb.fit(X_train, y_train)

# Оценка
best_xgb = random_search_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

print("\nXGBoost Results:")
print(f"MAE: {mean_absolute_error(y_test, y_pred_xgb):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_xgb):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_xgb)):.4f}")
print(f"R2: {r2_score(y_test, y_pred_xgb):.4f}")
print(f"Best params: {random_search_xgb.best_params_}")

## <font color='#11a642' size='5'> Построение модели бустинга **LightGBM**. Гиперпараметры подберите либо вручную, либо с помощью GridSearchCV или RandomizedSearchCV


- необходимо ли обрабатывать категориальные признаки заранее?
- необходимо ли обрабатывать пропуски?

In [ ]:
# ваш код
lgbm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(random_state=42, n_jobs=-1, verbosity=-1))
])

param_dist_lgbm = {
    'regressor__n_estimators': [100, 200],
    'regressor__learning_rate': [0.1, 0.2],
    'regressor__num_leaves': [31, 62],
    'regressor__feature_fraction': [0.8, 1.0]
}

print("Начинаем RandomizedSearch для LightGBM...")
random_search_lgbm = RandomizedSearchCV(
    lgbm_pipeline, 
    param_dist_lgbm, 
    n_iter=8, 
    cv=3, 
    scoring='neg_mean_squared_error', 
    random_state=42, 
    n_jobs=-1,
    verbose=1
)

random_search_lgbm.fit(X_train, y_train)

# Оценка
best_lgbm = random_search_lgbm.best_estimator_
y_pred_lgbm = best_lgbm.predict(X_test)

print("\nLightGBM Results:")
print(f"MAE: {mean_absolute_error(y_test, y_pred_lgbm):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_lgbm):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lgbm)):.4f}")
print(f"R2: {r2_score(y_test, y_pred_lgbm):.4f}")
print(f"Best params: {random_search_lgbm.best_params_}")

## <font color='#11a642' size='5'> Построение модели бустинга **Catboost**. Гиперпараметры подберите либо вручную, либо с помощью GridSearchCV или RandomizedSearchCV


- необходимо ли обрабатывать категориальные признаки заранее?
- необходимо ли обрабатывать пропуски?

In [ ]:
# ваш код
# Подготовим данные специально для CatBoost
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

# Обработаем пропуски только для числовых признаков
for col in num_features:
    X_train_cat[col].fillna(X_train_cat[col].median(), inplace=True)
    X_test_cat[col].fillna(X_train_cat[col].median(), inplace=True)

# Для категориальных признаков CatBoost обработает сам
for col in cat_features:
    X_train_cat[col].fillna('Unknown', inplace=True)
    X_test_cat[col].fillna('Unknown', inplace=True)

# Определяем индексы категориальных признаков
cat_features_indices = [X_train_cat.columns.get_loc(c) for c in cat_features]

# Простая модель CatBoost без пайплайна
catboost_model = CatBoostRegressor(
    iterations=200,  
    learning_rate=0.1,
    depth=6,
    cat_features=cat_features_indices,
    random_state=42,
    verbose=50  
)

print("Начинаем обучение CatBoost...")
catboost_model.fit(X_train_cat, y_train)

# Оценка
y_pred_cat = catboost_model.predict(X_test_cat)

print("\nCatBoost Results:")
print(f"MAE: {mean_absolute_error(y_test, y_pred_cat):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_cat):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_cat)):.4f}")
print(f"R2: {r2_score(y_test, y_pred_cat):.4f}")

## <font color='#11a642' size='5'> Выводы:

### <font color='#11a642' size='4'>
- Какая модель оказалась более стабильной и лучше по метрикам?
- Какая модель требует меньше дополнительных обработок данных?
- Какую модель вы выбираете и почему?

- По результатам оценки метрик (MAE, MSE, R2) лучшей моделью оказалась CatBoost.
- CatBoost требует меньше всего предварительной обработки данных, так как он автоматически обрабатывает категориальные признаки и устойчив к пропускам.
- CatBoost, так как он показал наилучшие результаты по метрикам, требует минимальной предварительной обработки данных и обладает хорошей устойчивостью к переобучению. Также CatBoost имеет встроенную обработку категориальных признаков, что упрощает процесс моделировани